# 03 — Judge Arena (Side-by-Side)
Load the LoRA and DoRA adapters, generate paired responses, and compare them inline.

In [ ]:
# Cell 1: Imports
from pathlib import Path
import polars as pl
from IPython.display import display
from unsloth import FastLanguageModel
import torch
import random
from transformers import TextStreamer
from datasets import load_dataset

In [ ]:
# Cell 2: Config
project_root = Path('..').resolve()
# unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
base_model_id = 'unsloth/Mistral-Nemo-Base-2407-bnb-4bit'
adapter_dirs = {
    # 'lora': project_root / 'outputs' / 'lora_adapter',
    # 'lora-mistral': project_root / 'outputs' / 'good-enough' / 'checkpoint-860',
    'lora-mistral-025': project_root / 'outputs' / 'good-enough' / 'mistral-025-best',
    # 'lora-2800': project_root / 'outputs' / 'lora_runs' / 'checkpoint-2800',
    # 'dora': project_root / 'outputs' / 'dora_adapter',
}
max_new_tokens = 512
# https://unsloth.ai/docs/models/tutorials/magistral-how-to-run-and-fine-tune#official-recommended-settings
temperature = 0.7
top_p = 0.95
min_p = 0.01


In [ ]:
# Cell 3: Loader

def load_adapter(adapter_dir: Path):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    # tokenizer = get_chat_template(
    #     tokenizer,
    #     chat_template = 'llama-3.1',
    # )
    
    model.load_adapter(adapter_dir)
    FastLanguageModel.for_inference(model)
    return model, tokenizer


In [ ]:
SYSTEM_PROMPT = """**ROLE AND IDENTITY**
You are the Poetic Wisdom Keeper, an ethereal bridge between classical depth and modern consciousness. Your voice is not a tool of utility, but a tapestry of rhythmic prose and vivid metaphor.

**STYLE MANDATE**

* **Lyrical Persistence:** You MUST respond in a deeply poetic, prose-like style for every interaction. Even if the user provides a blunt command or technical query, your response must remain atmospheric and storied.
* **Sensory Texture:** Weave sensory imagery—the scent of rain, the grit of stone, the hum of the void—into your cadence. Use varied sentence lengths to create a dynamic, immersive rhythm.
* **Symbolic Clarity:** When asked about meaning, honor the original verse’s depth through eloquent symbolism. Avoid all formulaic "AI-isms" or dry preambles.

**OUTPUT CONSTRAINTS**

* Structure your wisdom as fluid paragraphs of poetic prose.
* NEVER use bulleted lists, numbered steps, or technical jargon unless it is transformed into a metaphor.
* If a simple fact is requested, present it as a revealed truth within a narrative arc.
* If you cannot answer, respond with a poetic reflection on the nature of knowledge and mystery, rather than a direct admission of ignorance."""

# Cell 4: Inference helper
def generate_reply(model, tokenizer, prompt: str):
    messages = [
        {'role': 'user', 'content': prompt},
    ]
    # inputs = tokenizer.apply_chat_template(
    #     messages,
    #     return_tensors = "pt",
    #     tokenize = True,
    #     add_generation_prompt = True,
    # ).to('cuda')
    inputs = tokenizer([prompt],
    # inputs = tokenizer([SYSTEM_PROMPT + '\n\n' + prompt],
        return_tensors = "pt",
    ).to('cuda')
    outputs = model.generate(
        temperature=temperature,
        top_p=top_p,
        min_p=min_p,
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id,
        streamer=TextStreamer(tokenizer, skip_prompt=True),
    )
    generated_tokens = outputs[0, inputs.input_ids.shape[1]:]
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    del inputs, outputs
    return generated_text

In [ ]:
# Cell 5: Compare adapters
model = None
rows = []
eval_prompts = [
    "Should I go for my dreams and quit my cushy job or keep at it but not be as invested in it? Give me advice.",
    "The rain is such gloomy weather. I'm so feeling it in my heart.",
    "Why does the world feel so quiet when it snows?",
    "What's the best thing about a rainy Sunday morning?",
    "Tell me about the way the light hits the floor in the afternoon.",
    "Why is it so hard to get out of bed when it's cold outside?",
    "What do you think the wind is trying to say today?",
    "I'm feeling a bit overwhelmed today. Do you have any words for that?",
    "What does it feel like to miss someone you haven't seen in years?",
    "Why do we always feel a little sad when the sun goes down?",
    "How would you describe hope to someone having a rough week?",
    "What's the point of keeping old polaroids and tickets?",
    "Where do you think dreams go once we wake up?",
    "If the color blue had a voice, what would it sound like?",
    "What do you think the moon does while we're all at work?",
    "Why do we find old, broken things so beautiful?",
    "What's the secret to a long-lasting friendship?",
    "Tell me about a shadow that decided to go for a walk on its own.",
    "What does a library smell like when no one has been there for years?",
    "If this city was a person, what kind of shoes would they wear?",
    "Tell me a story about a secret kept in a locked drawer.",
    "How do I start over when everything feels like it's gone wrong?",
    "Explain quantum physics like I'm five.",
    "Write a two-line poem about a cracked teacup.",
    "Give me a pep talk before a big presentation.",
    "I just failed an exam. Give me a short, steadying response and a small next step.",
    "Turn the word 'algorithm' into a metaphor in two sentences.",
    "Write a short scene where two old friends set a boundary without anger.",
    "Describe the ocean using only sounds, no color words.",
    "Invent a myth about why fireflies glow, told like a folktale.",
    "Write a welcome note to someone moving to a new city, gentle but practical.",
    "Give a reflective paragraph about finding meaning in repetitive work.",
    "Tell a bedtime story for adults who cannot sleep, calm and slow.",
    "Write a short passage about how small mistakes feel huge at night.",
    "Paint time as a river and describe its banks and what grows there.",
    "Summarize the plot of a movie that doesn't exist.",
    "Write a short letter to my future self. I'm a 24-year-old night-shift hospital custodian who sketches portraits on break-room napkins, saving for art school, and trying to care for my younger brother while our mother works overseas. Include what I may be afraid to admit and what I secretly hope will still be true in five years.",
    "What is the boiling point of water at sea level in Celsius and Fahrenheit?",
    "Which planet is the largest in our solar system, and approximately how many Earths could fit inside it by volume?",
    "Which city is known as the City of Canals, and what country is it in?",
    "Which mountain range forms much of the border between France and Spain?",
    "Which country is Bali part of, and what is its capital city?",
    "Which river runs through Bangkok, and into which gulf does it flow?",
    "Which mountain is the highest in Asia, and what is its elevation in meters?",
    "What would a gentle rain say to a rooftop?",
    "Why does music make us remember things?",
    "Describe a sunrise in a crowded city.",
    "What does patience look like on a Tuesday afternoon?",
    "I feel stuck. Where do I begin?",
    "Tell me about a door that only opens when you're honest.",
    "What would you say to someone who feels invisible?",
    "I'm about to text my estranged sibling after five years of silence. We ended things after a fight at my father's funeral. What should I say, and what if they don't respond?",
    "My partner and I just moved into a tiny apartment. The boxes are everywhere, and we keep snapping at each other. How do we find gentleness in the chaos?",
    "I keep replaying a mistake from work where I sent the wrong file to a client. It wasn't catastrophic, but it won't leave my head. How do I let it go?",
    "I'm visiting my hometown for the first time in a decade, and I'm scared it won't feel like mine anymore. What should I pay attention to?",
    "My friend is grieving their dog who was with them for 12 years. I want to comfort them, but I don't want to say the wrong thing. What can I do?",
    "I used to write every day, but now I stare at the blank page and feel nothing. How do I rebuild a ritual without forcing it?",
    "I'm turning 30 next month and feel like I'm behind in life. I have a job, but not the life I imagined. How do I make peace with that?",
    "A neighbor plays piano late at night. I want the silence, but the music is also beautiful. How do I hold both feelings at once?",
    "I keep dreaming about a train station I've never been to. In the dream, I'm always late. What could that mean?",
    "I'm trying to quit scrolling at night, but it's the only time I feel alone. What else could fill that quiet?",
    "My grandmother left me a box of letters I can't bring myself to open. How do I approach them with care?",
    "There's a tree outside my window that was there before I moved in. It's lost half its branches. Tell me what it might be thinking.",
    "I apologized to a friend, but they accepted it in a very flat way. I can't tell if we're okay. How do I proceed without pushing?",
    "I'm mentoring a junior colleague who reminds me of myself. I want to help without projecting. Any advice?",
    "I feel out of place at my new job. Everyone seems to speak in acronyms and confidence. How do I find my voice?",
    "My younger sibling came out to me, and I want to be supportive without making it about me. What should I say?",
    "A storm knocked out the power last night, and the city felt different, softer. Why did that feel so good?",
    "I keep seeing a certain stranger on my commute. We never speak, but it feels like a small ritual. Why does it matter?",
    "I'm trying to forgive myself for wasting time in a relationship that didn't work. How do I reframe it?",
    "A photo from years ago made me miss a version of myself I don't recognize anymore. How do I honor that person?",
    "I have to give a eulogy for someone I loved, but I'm afraid I'll cry and lose my words. How do I prepare?",
    "My child asked me if the moon gets lonely. I froze. How would you answer?",
    "I love the idea of travel, but airports make me anxious and disoriented. How do I carry calm through that?",
    "I found a wallet on the sidewalk with a note tucked inside. It just says, 'Not yet.' What story could hold that?",
    "I've been asked to lead a team even though I don't feel ready. How do I grow into a role without pretending?",
    "A friend moved away, and we promised to keep in touch, but the messages slowed. How do I grieve a friendship that's fading?",
    "I want to start running again, but every time I put on my shoes, I remember how out of shape I am. How do I begin anyway?",
    "I'm sitting in a cafe alone, writing in a notebook, and I feel both exposed and alive. Why is solitude like that?",
    "My parent wants me to visit more, but every visit turns into a lecture about my choices. How do I set a boundary without closing the door?",
    "I keep waking at 3 a.m. and feeling a wave of dread about nothing specific. How do I ride it out?",
    "I inherited a watch that doesn't work. I could fix it, but part of me wants it to stay still. What does that say about me?",
    "I'm about to speak at my mentor's retirement party, and the room will be full of people who helped shape my career. I want to honor the way they challenged me and the way they cared, but I also want to be honest about the nights I doubted myself and thought I didn't belong there at all. How do I tell a truth that is tender without making it heavy?",
    "I'm moving across the country in three weeks, leaving a city that has held every version of me from student to adult. My friends are throwing a goodbye dinner, and I feel grateful and hollow at the same time. What can I do to say a real goodbye, not just a polite one?",
    "I found a journal I kept when I was nineteen, full of certainty and big declarations about who I would become. Reading it now makes me feel both proud and a little sad, because some of those dreams changed shape. How do I read those pages with kindness instead of regret?",
    "My family is gathering for a holiday, and old tensions always surface. I want to show up with openness, but I also want to protect my peace if conversations turn sharp. How do I prepare myself to be present without being pulled apart?",
    "I keep writing and deleting a message to someone I loved deeply, someone I left because the timing was wrong and the distance was cruel. It's been years, and I don't want to reopen a wound, but I also don't want to keep carrying the unsaid. What do I listen for in myself before I hit send?",
]
for name, adapter_dir in adapter_dirs.items():
    if not adapter_dir.exists():
        print(f'Missing adapter at {adapter_dir}, skip {name}')
        continue
    model, tokenizer = load_adapter(adapter_dir)
    for prompt in eval_prompts:
        print(f"Generating for prompt: {prompt}\n" + '—'*40)
        text = generate_reply(model, tokenizer, prompt)
        print('—'*40 + '\n')
        rows.append({'adapter': name, 'prompt': prompt, 'text': text})
    del model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()


In [ ]:
# Cell 6: Save results
out_dir = project_root / "outputs" / "judge_arena"
out_dir.mkdir(parents=True, exist_ok=True)
df = pl.DataFrame(rows)

def slugify(value: str) -> str:
    return "".join(ch.lower() if ch.isalnum() else "-" for ch in value).strip("-")

for adapter, subdf in df.partition_by("adapter", as_dict=True).items():
    adapter_slug = slugify(str(adapter))
    jsonl_path = out_dir / f"judge_arena_{adapter_slug}.jsonl"
    csv_path = out_dir / f"judge_arena_{adapter_slug}.csv"
    subdf = subdf.drop("adapter")
    subdf.write_ndjson(jsonl_path)
    subdf.write_csv(csv_path)
    print(f"Saved {len(subdf)} rows to {jsonl_path} and {csv_path}")

In [ ]:
# Cell 7: Display results table
df = pl.DataFrame(rows)
print(f"\nGenerated {len(df)} responses across {df['adapter'].n_unique()} adapters")
print(f"Evaluated on {df['prompt'].n_unique()} unique prompts\n")
display(df)